In [1]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
val bucketname = "unibo-bd2324-dcohen"

val path_apps = "s3a://"+bucketname+"/datasets/project/App_ID_Info.csv"
val path_players = "s3a://"+bucketname+"/datasets/project/Player_Summaries.csv"
val path_friends = "s3a://"+bucketname+"/datasets/project/Friends.csv"
val path_games_full = "s3a://"+bucketname+"/datasets/project/Games.csv"
val path_games = "s3a://"+bucketname+"/datasets/project/Games20Percent.csv"
val path_games_daily = "s3a://"+bucketname+"/datasets/project/Games_Daily.csv"
val path_games_info = "s3a://"+bucketname+"/datasets/project/Games_Info.csv"
val path_achievement = "s3a://"+bucketname+"/datasets/project/Achievement_Percentages.csv"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1697891147468_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2324-dcohen
path_apps: String = s3a://unibo-bd2324-dcohen/datasets/project/App_ID_Info.csv
path_players: String = s3a://unibo-bd2324-dcohen/datasets/project/Player_Summaries.csv
path_friends: String = s3a://unibo-bd2324-dcohen/datasets/project/Friends.csv
path_games_full: String = s3a://unibo-bd2324-dcohen/datasets/project/Games.csv
path_games: String = s3a://unibo-bd2324-dcohen/datasets/project/Games20Percent.csv
path_games_daily: String = s3a://unibo-bd2324-dcohen/datasets/project/Games_Daily.csv
path_games_info: String = s3a://unibo-bd2324-dcohen/datasets/project/Games_Info.csv
path_achievement: String = s3a://unibo-bd2324-dcohen/datasets/project/Achievement_Percentages.csv
res3: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1697891147468_0001/


In [120]:
import java.text.SimpleDateFormat
import java.util.Calendar
import scala.util.{Try, Success, Failure}

case class App(appid: Long, title: String, typ: String, price: Double, releaseYear: Int, rating: Int, requiredAge: Int, isMultiplayer: Int)
case class PlayerSummary(steamid: Long, personaname: String, lastlogoffYear: Int, realname: Option[String], primaryclanid: Long, timecreatedYear: Int, loccountrycode: Option[String], locstatecode: Option[String])
case class GameInfo(appid: Long, developer: String, publisher: String, genre: String)
case class Game(steamid: Long, appid: Long, playtimeTwoWeek: Int, playtimeForever: Int)
case class Friend(steamidA: Long, steamidB: Long, friendSinceYear: Int)
case class Achievement(appid: Long, name: String, percentage: Float)

case class AppValues(title: String, typ: String, price: Double, releaseYear: Int, rating: Int, requiredAge: Int, isMultiplayer: Int)
case class PlayerSummaryValues(personaname: String, lastlogoffYear: Int, realname: Option[String], primaryclanid: Long, timecreatedYear: Int, loccountrycode: Option[String], locstatecode: Option[String])
case class GameInfoValues(developer: String, publisher: String, genre: String)
case class GameValues(playtimeTwoWeek: Int, playtimeForever: Int)
case class FriendValues(friendSinceYear: Int)
case class AchievementValues(name: String, percentage: Float)

object SteamGamesParser {

  val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val quotes = "\""

    /** Convert from date string "yyyy-MM-dd HH:mm:ss" to year (Int) */
    def yearFromTimestamp(timestamp: String): Int = {
      val dateFormat = new SimpleDateFormat("yyyy-MM-dd HH:mm:ss")
      val date = dateFormat.parse(timestamp)
      val cal = Calendar.getInstance()
      cal.setTime(date)
      cal.get(Calendar.YEAR)
    }
    


    def cleanField(field: String): String = field.replace(quotes, "").trim

    def safelyConvert[A](str: String)(conversion: String => A): Option[A] = {
      Try(conversion(str)) match {
        case Success(value) => Some(value)
        case Failure(e) => 
          println(s"Error converting value '$str': ${e.getMessage}")
          None
      }
    }

    /** PARSING **/
    
    def parseAppLine(line: String): Option[App] = {
      val fields = line.split(",").map(cleanField)

      fields match {
        case Array(
                appid, 
                title, 
                typ, 
                price, 
                releaseDate, 
                rating, 
                requiredAge, 
                isMultiplayer) =>
          for {
            appidLong        <- safelyConvert(appid)(_.toLong)
            priceDouble      <- safelyConvert(price)(_.toDouble)
            releaseYear      <- safelyConvert(releaseDate)(yearFromTimestamp)
            ratingInt        <- safelyConvert(rating)(_.toInt)
            requiredAgeInt   <- safelyConvert(requiredAge)(_.toInt)
            isMultiplayerInt <- safelyConvert(isMultiplayer)(_.toInt)
          } yield App(
            appidLong, 
            title, 
            typ, 
            priceDouble, 
            releaseYear, 
            ratingInt, 
            requiredAgeInt, 
            isMultiplayerInt
        )
        case _ => None
      }
    }
    
    def parsePlayerSummaryLine(line: String): Option[PlayerSummary] = {
      val fields = line.split(",").map(cleanField)
      fields match {
        case Array(
                steamid, 
                personaname, 
                profileurl, 
                avatar, 
                avatarmedium, 
                avatarfull, 
                personastate, 
                communityvisibilitystate, 
                profilestate, 
                lastlogoff, 
                commentpermission, 
                realname, 
                primaryclanid, 
                timecreated, 
                gameid, 
                gameserverip, 
                gameextrainfo, 
                cityid, 
                loccountrycode, 
                locstatecode, 
                loccityid,
                dateretrieved) =>
          for {
            steamidLong        <- safelyConvert(steamid)(_.toLong)
            primaryclanidLong  <- safelyConvert(primaryclanid)(_.toLong)
            lastlogoffYear     <- safelyConvert(lastlogoff)(yearFromTimestamp)
            timecreatedYear    <- safelyConvert(timecreated)(yearFromTimestamp)
            loccityidInt       <- safelyConvert(loccityid)(_.toInt)
          } yield PlayerSummary(
            steamidLong, 
            personaname,
            lastlogoffYear, 
            Option(realname).orElse(Some("Unknown")), 
            primaryclanidLong, 
            timecreatedYear, 
            Option(loccountrycode).orElse(Some("Unknown")), 
            Option(locstatecode).orElse(Some("Unknown"))
          )
        case _ => None
      }
    }
    
    def parseGameInfoLine(line: String): Option[GameInfo] = {
      val fields = line.split(",").map(cleanField)

      fields match {
        case Array(
                appid, 
                developer, 
                publisher, 
                genre) =>
          for {
            appidLong  <- safelyConvert(appid)(_.toLong)
          } yield GameInfo(
            appidLong, 
            developer, 
            publisher, 
            genre
        )
        case _ => None
      }
    }

    def parseGameLine(line: String): Option[Game] = {
      val fields = line.split(",").map(cleanField)

      fields match {
        case Array(
                steamid, 
                appid, 
                playtimeTwoWeek,
                playtimeForever,
                dateretrieved) =>
          for {
            steamidLong  <- safelyConvert(steamid)(_.toLong)
            appidLong  <- safelyConvert(appid)(_.toLong)
            playtimeTwoWeekInt  <- safelyConvert(appid)(_.toInt)
            playtimeForeverInt  <- safelyConvert(appid)(_.toInt)
          } yield Game(
            steamidLong, 
            appidLong, 
            playtimeTwoWeekInt, 
            playtimeForeverInt
        )
        case _ => None
      }
    }
    
    def parseFriendLine(line: String): Option[Friend] = {
      val fields = line.split(",",-1).map(cleanField) 
        fields match {
          case Array(
              steamid_a,
              steamid_b,
              relationship,
              friendSince,
              dataretrived,
              lccTag) =>
            for {
              steamidALong     <- safelyConvert(steamid_a)(_.toLong)
              steamidBLong     <- safelyConvert(steamid_b)(_.toLong)
              friendSinceYear  <- safelyConvert(friendSince)(yearFromTimestamp)
            } yield Friend(
              steamidALong,
              steamidBLong,
              friendSinceYear
          )
          case _ => None
        }
    }
    
    def parseAchievementLine(line: String): Option[Achievement] = {
      val fields = line.split(",").map(cleanField) 
        fields match {
          case Array(
              appid,
              name,
              percentage,
              _*) =>
            for {
              appidLong           <- safelyConvert(appid)(_.toLong)
              percentageFloat     <- safelyConvert(percentage)(_.toFloat)
            } yield Achievement(
              appidLong,
              name,
              percentageFloat
          )
          case _ => None
        }
    }
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.text.SimpleDateFormat
import java.util.Calendar
import scala.util.{Try, Success, Failure}
defined class App
defined class PlayerSummary
defined class GameInfo
defined class Game
defined class Friend
defined class Achievement
defined class AppValues
defined class PlayerSummaryValues
defined class GameInfoValues
defined class GameValues
defined class FriendValues
defined class AchievementValues
defined object SteamGamesParser


In [77]:
val rddApps = sc.textFile(path_apps).flatMap(SteamGamesParser.parseAppLine)
val rddPlayerSummaries = sc.textFile(path_players).flatMap(SteamGamesParser.parsePlayerSummaryLine)
val rddGamesInfo = sc.textFile(path_games_info).flatMap(SteamGamesParser.parseGameInfoLine)
val rddGames = sc.textFile(path_games).flatMap(SteamGamesParser.parseGameLine).sample(false, 0.03) //changeIt
val rddFriends = sc.textFile(path_friends).flatMap(SteamGamesParser.parseFriendLine).sample(false, 0.2) //changeIt
val rddAchievements = sc.textFile(path_achievement).flatMap(SteamGamesParser.parseAchievementLine)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddApps: org.apache.spark.rdd.RDD[App] = MapPartitionsRDD[280] at flatMap at <console>:101
rddPlayerSummaries: org.apache.spark.rdd.RDD[PlayerSummary] = MapPartitionsRDD[283] at flatMap at <console>:101
rddGamesInfo: org.apache.spark.rdd.RDD[GameInfo] = MapPartitionsRDD[286] at flatMap at <console>:101
rddGames: org.apache.spark.rdd.RDD[Game] = PartitionwiseSampledRDD[290] at sample at <console>:101
rddFriends: org.apache.spark.rdd.RDD[Friend] = PartitionwiseSampledRDD[294] at sample at <console>:101
rddAchievements: org.apache.spark.rdd.RDD[Achievement] = MapPartitionsRDD[297] at flatMap at <console>:101


In [126]:
import org.apache.spark.storage.StorageLevel

// val rddAppsCached = rddApps.cache()
// val rddPlayerSummariesCached = rddApps.cache()
// val rddGamesInfoCached = rddGamesInfo.cache()
// val rddGamesCached = rddGames.persist(StorageLevel.MEMORY_ONLY)
// val rddFriendsCached = rddFriends.persist(StorageLevel.MEMORY_ONLY)
// val rddAchievementsCached = rddAchievements.cache()

val rddAppsCached = rddApps.map { app => 
  (app.appid, AppValues(app.title, app.typ, app.price, app.releaseYear, app.rating, app.requiredAge, app.isMultiplayer))
}.cache()

val rddPlayerSummariesCached = rddPlayerSummaries.map { player => 
  (player.steamid, PlayerSummaryValues(player.personaname, player.lastlogoffYear, player.realname, player.primaryclanid, player.timecreatedYear, player.loccountrycode, player.locstatecode))
}.cache()

val rddGamesInfoCached = rddGamesInfo.map { gameInfo => 
  (gameInfo.appid, GameInfoValues(gameInfo.developer, gameInfo.publisher, gameInfo.genre))
}.cache()

val rddGamesCached = rddGames.map { game => 
  ((game.steamid, game.appid), GameValues(game.playtimeTwoWeek, game.playtimeForever))
}.persist(StorageLevel.MEMORY_ONLY)

val rddFriendsCached = rddFriends.map { friend => 
  ((friend.steamidA, friend.steamidB), FriendValues(friend.friendSinceYear))
}.persist(StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.storage.StorageLevel
rddAppsCached: org.apache.spark.rdd.RDD[(Long, AppValues)] = MapPartitionsRDD[500] at map at <console>:137
rddPlayerSummariesCached: org.apache.spark.rdd.RDD[(Long, PlayerSummaryValues)] = MapPartitionsRDD[501] at map at <console>:130
rddGamesInfoCached: org.apache.spark.rdd.RDD[(Long, GameInfoValues)] = MapPartitionsRDD[502] at map at <console>:130
rddGamesCached: org.apache.spark.rdd.RDD[((Long, Long), GameValues)] = MapPartitionsRDD[503] at map at <console>:130
rddFriendsCached: org.apache.spark.rdd.RDD[((Long, Long), FriendValues)] = MapPartitionsRDD[504] at map at <console>:130


In [79]:
// Total number of games sessions 
println(s"Total number of games sessions: ${rddGamesCached.count()}")

// Total number of games
// (Possibile candidata broadcast)
println(s"Total number of games: ${rddAppsCached.count()}")

// Total number of players
// (Possibile candidata broadcast)
println(s"Total number of players: ${rddPlayerSummariesCached.count()}")

// Total number of friends connections
println(s"Total number of friends connections: ${rddFriendsCached.count()}")

// Total number of game informations
// (Possibile candidata broadcast, solo se le informazioni sono molto leggere)
println(s"Total number of game informations: ${rddGamesInfoCached.count()}")

// Total number of achievements
println(s"Total number of achievements: ${rddAchievementsCached.count()}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of games sessions: 932897
Total number of games: 17606
Total number of players: 17606
Total number of friends connections: 1984469
Total number of game informations: 43306
Total number of achievements: 156188


In [143]:
// Aggrega il tempo di gioco totale per ogni gioco e stampa il primo (NON OTTIMIZZATA)

val totalPlaytime = rddGamesCached.
    map{ case ((steamid, appid), gameValues) => (appid, gameValues.playtimeForever) }.
    reduceByKey(_ + _)

val AppsKeyWithTitle = rddAppsCached.
    mapValues(_.title)

val totalPlaytimeWithNames = totalPlaytime.
    join(AppsKeyWithTitle).
    sortBy(_._2._1, ascending = false)

val mostPlayedGame = totalPlaytimeWithNames.
    sortBy(_._2._1, ascending = false).
    first()

println(s"\n\nThe most played game is ${mostPlayedGame._2._2} with ${mostPlayedGame._2._1} minutes of total playtime.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

totalPlaytime: org.apache.spark.rdd.RDD[(Long, Int)] = ShuffledRDD[564] at reduceByKey at <console>:131
AppsKeyWithTitle: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[565] at mapValues at <console>:129
totalPlaytimeWithNames: org.apache.spark.rdd.RDD[(Long, (Int, String))] = MapPartitionsRDD[576] at sortBy at <console>:131
mostPlayedGame: (Long, (Int, String)) = (246700,(448007200,Strike Vector))


The most played game is Strike Vector with 448007200 minutes of total playtime.


In [151]:
// Aggrega il tempo di gioco totale per ogni gioco e stampa il primo (OTTIMIZZATA)

// Tempo di gioco totale per ogni utente
val totalPlaytime = rddGamesCached.
    mapValues(_.playtimeForever).
    reduceByKey(_ + _)

val AppsKeyWithTitle = rddAppsCached.
    mapValues(_.title)

// 1. Convertire l'RDD a una struttura dati (Map)
val AppsKeyWithTitleMap: Map[Long, String] = AppsKeyWithTitle.
    collect().
    toMap

val broadcastedAppsKeyWithTitle = sc.broadcast(AppsKeyWithTitleMap)

// Esegui il mapping del primo RDD per includere il nome del gioco usando la mappa trasmessa
val totalPlaytimeWithNames = totalPlaytime.map { case ((steamid,appid), time) =>
  val gameName = broadcastedAppsKeyWithTitle.value.getOrElse(appid, "Unknown")
  ((appid, gameName), time)
}

val mostPlayedGame = GameMorePlayed.
    reduce((a, b) => if (a._2._1 > b._2._1) a else b)

println(s"The most played game is ${mostPlayedGame._2._2} with ${mostPlayedGame._2._1} minutes of total playtime.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

totalPlaytime: org.apache.spark.rdd.RDD[((Long, Long), Int)] = ShuffledRDD[601] at reduceByKey at <console>:132
AppsKeyWithTitle: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[602] at mapValues at <console>:129
AppsKeyWithTitleMap: scala.collection.immutable.Map[Long,String] = Map(42890 -> Sword of the Stars: Complete Collection, 221020 -> Towns, 428140 -> TowerClimb - Extended Original Soundtrack, 402840 -> Crusaders of the Lost Idols, 204308 -> Awesomenauts - Hotrod Derpl Skin, 42900 -> Hearts of Iron III: Semper Fi, 446870 -> Glitchrunners, 421166 -> MX vs. ATV Supercross Encore - 2 Stroke 4 Pack, 90410 -> Blue Toad Murder Files?: The Mysteries of Little Riddle Demo, 259620 -> 3079 -- Block Action RPG, 325330 -> Muvee Reveal 11, 349680 -> Supercharged Robot VULKAISER, 318070 -> Happy Wars - Happy Tickets - Value Pack A, 202200 -> Galactic Civilizations? II: Ultimate Edition, 344130 -> Cataegis : The White Wind, 202310 -> Ridge Racer? Unbounded, 206104 -> Rocksmith - Th

In [152]:
// Caching dei dati utilizzati più volte
// Estrai solo il loccountrycode da ciascun player e memorizza in cache il risultato
val cachedPlayerSummaries = rddPlayerSummariesCached.
    mapValues(_.loccountrycode.getOrElse("Unknown")).
    cache()

// Contiamo il numero di amici per ogni steamid
// Ogni coppia di amici viene decomposta in due elementi: uno per ciascun amico
val friendCounts = rddFriendsCached.
    flatMap{ case ((steamidA, steamidB), _) => Seq((steamidA, 1), (steamidB, 1)) }.
    reduceByKey(_ + _)

// Aggiungi il countryCode a ciascun conteggio di amici
val friendCountsByCountry = friendCounts.join(cachedPlayerSummaries).
    mapValues { case (count, countryCode) => (countryCode, count) }

// Calcola la media del numero di amici per countryCode
// Utilizzo di aggregateByKey per calcolare sia la somma che il conteggio
val sumAndCountByCountry = friendCountsByCountry.
    aggregateByKey((0, 0))(
        (acc, value) => (acc._1 + value._2, acc._2 + 1),
        (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)
    )

// Calcolo della media
val avgByCountry = sumAndCountByCountry.
    mapValues{ case (sum, count) => if (count > 0) sum / count.toDouble else 0.0 }

// Stampa la media del numero di amici per ciascun paese
avgByCountry.collect().foreach { case (countryCode, avg) => println(s"Country: $countryCode, Average Friends: $avg") }


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:
avgByCountry.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res709: Array[(Long, Double)] = Array((76561197965200157,2.0), (76561197963724621,1.0), (76561197967189641,2.0), (76561197971208901,1.0), (76561197960425989,13.0), (76561197962555949,6.0), (76561197966829085,3.0), (76561197967462501,1.0), (76561197961272625,4.0), (76561197970833813,4.0))
